In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
import os

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
def find_csv_files(folder_path):
    csv_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

In [4]:
def show(df, limit=10):
    return df.toPandas().head(limit)

In [5]:
folder = r"C:\Development\ultimateNakMuay\data\processed"
files = find_csv_files(folder)

In [6]:
files = files[2:]

In [7]:
organizations = ["ufc", "onefc", "bellator", "glory"]

organizations = {
    k:{
        "event": spark.read.csv(list(filter(lambda x: "event" in x and k in x, files))[0], header=True),
        "result": spark.read.csv(list(filter(lambda x: "result" in x and k in x, files))[0], header=True),
        }
    for k in organizations
}

In [8]:
def get_data(org):
    return organizations[org]["event"], organizations[org]["result"]

In [9]:
event, result = get_data("glory")

In [30]:
event = event.withColumn("ejc1", F.regexp_replace(F.col("event"), ":", ""))\
    .withColumn("ejc2", F.element_at(F.split(F.col("event"), ":"), 1))

In [28]:
result = result.withColumn("rjc1", F.regexp_replace(F.col("event_name"), ":", ""))

In [31]:
join_conditions = (
    (event.event == result.event_name)
    | (event.ejc1 == result.event_name)
    | (event.ejc1 == result.rjc1)
    | (event.ejc2 == result.rjc1)
)

show(
    event.join(result, join_conditions, "leftanti")
    .select(*event.columns[:3]).distinct(),
    1000
)

,event_num,event,date
0,14,2013 Road to Glory USA Heavyweight Tournament,2013-06-14
1,13,2013 Road to Glory USA 70 kg Tournament,2013-05-11
2,8,Road to Glory USA 77 kg Tournament,2013-03-22
3,46,Road to Glory UK 65 kg Tournament,2017-03-11
4,5,Road to Glory USA 95 kg Tournament,2013-02-02
5,6,Road to Glory USA 85 kg Tournament,2013-02-09
6,7,Road to Glory Japan 65 kg Tournament,2013-03-10


In [32]:
show(result.filter(F.col("event_name").like("%Tournament%")).select("event_name").distinct(), 100)

,event_name
